<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/HybridNCF_CBF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
place=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/tourism_with_id.csv")
rating=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/tourism_rating.csv")
user=pd.read_csv("drive/MyDrive/S3/Eksperimen/Data/user.csv")

In [4]:
place.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [5]:
place1=place[["Place_Id", "Place_Name", "Description", "City", "Price", "Rating"]]

In [6]:
place1.columns

Index(['Place_Id', 'Place_Name', 'Description', 'City', 'Price', 'Rating'], dtype='object')

In [7]:
place2=place1[place1["City"]=="Surabaya"]
place2

,Place_Id,Place_Name,Description,City,Price,Rating
391,392,Ekowisata Mangrove Wonorejo,Hutan Wisata Mangrove Surabaya merupakan wisat...,Surabaya,0,4.3
392,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
393,394,Air Mancur Menari,Jembatan Kenjeran dengan air mancur menarinya ...,Surabaya,35000,4.4
394,395,Taman Prestasi,Taman Prestasi Surabaya merupakan salah satu t...,Surabaya,0,4.6
395,396,Monumen Kapal Selam,"Monumen Kapal Selam, atau disingkat Monkasel, ...",Surabaya,15000,4.4
396,397,Taman Kunang-Kunang,Taman Kunang – Kunang di Surabaya ini mempunya...,Surabaya,0,4.4
397,398,Taman Buah Surabaya,Wisata Taman Buah Undaan di Surabaya adalah sa...,Surabaya,0,4.1
398,399,Taman Pelangi,Kalau pelangi biasanya ada di siang hari pasca...,Surabaya,0,4.5
399,400,Hutan Bambu Keputih,Surabaya menjadi kota metropolitan kedua setel...,Surabaya,0,4.3
400,401,Taman Keputran,"Ntah, mengapa nama taman ini disebut dengan ta...",Surabaya,0,4.3


In [8]:
place2=place2.reset_index(drop=True)
place2

,Place_Id,Place_Name,Description,City,Price,Rating
0,392,Ekowisata Mangrove Wonorejo,Hutan Wisata Mangrove Surabaya merupakan wisat...,Surabaya,0,4.3
1,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
2,394,Air Mancur Menari,Jembatan Kenjeran dengan air mancur menarinya ...,Surabaya,35000,4.4
3,395,Taman Prestasi,Taman Prestasi Surabaya merupakan salah satu t...,Surabaya,0,4.6
4,396,Monumen Kapal Selam,"Monumen Kapal Selam, atau disingkat Monkasel, ...",Surabaya,15000,4.4
5,397,Taman Kunang-Kunang,Taman Kunang – Kunang di Surabaya ini mempunya...,Surabaya,0,4.4
6,398,Taman Buah Surabaya,Wisata Taman Buah Undaan di Surabaya adalah sa...,Surabaya,0,4.1
7,399,Taman Pelangi,Kalau pelangi biasanya ada di siang hari pasca...,Surabaya,0,4.5
8,400,Hutan Bambu Keputih,Surabaya menjadi kota metropolitan kedua setel...,Surabaya,0,4.3
9,401,Taman Keputran,"Ntah, mengapa nama taman ini disebut dengan ta...",Surabaya,0,4.3


In [9]:
place2.columns

Index(['Place_Id', 'Place_Name', 'Description', 'City', 'Price', 'Rating'], dtype='object')

In [10]:
rating.columns

Index(['User_Id', 'Place_Id', 'Place_Ratings'], dtype='object')

In [11]:
user.columns

Index(['User_Id', 'Location', 'Age'], dtype='object')

In [12]:
data1=pd.merge(rating, place2, on="Place_Id")
data1.head()

,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Price,Rating
0,1,393,5,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
1,6,393,2,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
2,8,393,5,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
3,19,393,3,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4
4,27,393,3,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,Surabaya,0,4.4


In [13]:
data2=data1.rename({"User_Id":"user_id", "Place_Id":"place_id",  "Place_Ratings":"rating1", "Place_Name":"place_name", "Description":"description","Price":"price", "Rating":"rating2"}, axis=1)

In [14]:
data2.columns

Index(['user_id', 'place_id', 'rating1', 'place_name', 'description', 'City',
       'price', 'rating2'],
      dtype='object')

In [15]:
df=data2[["user_id","place_id", "place_name", "description", "price", "rating1", "rating2"]]

In [16]:
df.head()

,user_id,place_id,place_name,description,price,rating1,rating2
0,1,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,5,4.4
1,6,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,2,4.4
2,8,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,5,4.4
3,19,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,3,4.4
4,27,393,Taman Harmoni Keputih,Tempat tersebut ialah Taman Hatmoni Keputih Su...,0,3,4.4


In [17]:
# Menggabungkan rating menjadi satu nilai rata-rata
df['average_rating'] = df[['rating1', 'rating2']].mean(axis=1)

In [18]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['description']).toarray()

In [19]:
# Encoding categorical data
user_ids = df['user_id'].astype('category').cat.codes.values
place_ids = df['place_id'].astype('category').cat.codes.values

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(np.hstack((user_ids.reshape(-1,1), place_ids.reshape(-1,1), tfidf_matrix, df[['price']].values)), df['average_rating'], test_size=0.20, random_state=42)

In [20]:
# Input layers
user_input = Input(shape=(1,), dtype='int32')
place_input = Input(shape=(1,), dtype='int32')
price_input = Input(shape=(1,))
text_input = Input(shape=(tfidf_matrix.shape[1],))

# Embeddings
user_embedding = Embedding(input_dim=np.max(user_ids)+1, output_dim=10)(user_input)
place_embedding = Embedding(input_dim=np.max(place_ids)+1, output_dim=10)(place_input)

# Flatten the embeddings
user_vec = Flatten()(user_embedding)
place_vec = Flatten()(place_embedding)

# Concatenate all inputs
concat = Concatenate()([user_vec, place_vec, price_input, text_input])

# Neural network layers
dense = Dense(128, activation='relu')(concat)
dropout = Dropout(0.5)(dense)
output = Dense(1)(dropout)

model = Model(inputs=[user_input, place_input, price_input, text_input], outputs=output)
model.compile(optimizer=Adam(0.01), loss='mean_squared_error')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 10)                2930      ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 10)                460       ['input_2[0][0]']             
                                                                                              

In [21]:
model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3:]], y_train, epochs=100, batch_size=2, verbose=1)

# Predictions
predictions = model.predict([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3:]]).flatten()

# Metrics
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Confusion Matrix
predicted_classes = np.round(predictions)
cm = confusion_matrix(y_test.round(), predicted_classes, labels=[1, 2, 3, 4, 5])

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print("Confusion Matrix:\n", cm)

Epoch 1/100
420/420 [==============================] - 4s 7ms/step - loss: 1537653.7500
Epoch 2/100
420/420 [==============================] - 2s 5ms/step - loss: 7244.1626
Epoch 3/100
420/420 [==============================] - 2s 4ms/step - loss: 1310.8900
Epoch 4/100
420/420 [==============================] - 2s 4ms/step - loss: 524.0635
Epoch 5/100
420/420 [==============================] - 2s 4ms/step - loss: 102.1372
Epoch 6/100
420/420 [==============================] - 2s 4ms/step - loss: 53.1422
Epoch 7/100
420/420 [==============================] - 2s 4ms/step - loss: 31.2385
Epoch 8/100
420/420 [==============================] - 2s 6ms/step - loss: 17.5584
Epoch 9/100
420/420 [==============================] - 3s 7ms/step - loss: 11.4079
Epoch 10/100
420/420 [==============================] - 2s 4ms/step - loss: 8.4162
Epoch 11/100
420/420 [==============================] - 2s 4ms/step - loss: 6.8459
Epoch 12/100
420/420 [==============================] - 2s 4ms/step - loss: 